## Preprocessando SINAN raw_data

### Dependências

In [23]:
import pandas as pd
import plotly.express as px
import numpy as np

### Carregando dados

In [2]:
SINAN_RAW_PATH = './../data/raw_data/2. SINAN (violencia sexual)/DadosViolSexualDomestica_2013-18_SINAN.csv'

In [69]:
%%time

# Apenas para testes (ler apenas as primeiras nrows linhas)
nrows = 1000000

raw_dataframe = pd.read_csv(SINAN_RAW_PATH, nrows = nrows)

print('Número de colunas:', len(raw_dataframe.columns))
print('Número de linhas:', len(raw_dataframe))

C:\Users\Lucas\Anaconda3\envs\personal\lib\site-packages\decorator.py:231: DtypeWarning:

Columns (24,26,28,38,41,42,43,44,45,46,47,48,49,50,51,52,53,55,56,57,58,59,60,61,62,63,65,66,67,68,69,70,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,93,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,130,136,137,138) have mixed types.Specify dtype option on import or set low_memory=False.



Número de colunas: 158
Número de linhas: 1000000
Wall time: 15.4 s


### Agrupamento de variáveis

#### Relação com agressor

In [111]:
def get_agressor_relationship(dataframe):

    '''
    Retorna uma lista das relações das vítimas com os agressores.

    Ex: ['Pai', 'Desconhecido', ..., 'N/A']
    '''

    relationship_dict = {
        'REL_PAI': 'Pai',
        'REL_MAE': 'Mãe',
        'REL_PAD': 'Padrasto',
        'REL_CONJ': 'Cônjugue',
        'REL_EXCON': 'Ex-cônjuge',
        'REL_NAMO': 'Namorado/a',
        'REL_EXNAM': 'Ex-namorado/a',
        'REL_FILHO': 'Filho/a',
        'REL_DESCO': 'Desconhecido',
        'REL_IRMAO': 'Irmão/ã',
        'REL_CONHEC': 'Amigo/a',
        'REL_CUIDA': 'Cuidador/a',
        'REL_PATRAO': 'Chefe',
        'REL_INST': 'Relação Institucional',
        'REL_POL': 'Policial/Agente da lei',
        'REL_PROPRI': 'Próprio/a',
        'REL_OUTROS': 'Outra pessoa',
        'REL_MAD': 'Madrasta'
    }

    agressor_relationship = []

    for index in dataframe.index:

        case = dataframe.loc[index]

        found_rel = False

        for key in relationship_dict.keys():

            relantionship = relationship_dict[key]

            if case[key] == 1:

                agressor_relationship.append(relantionship)
                found_rel = True

                break

        if not found_rel:

            agressor_relationship.append('N/A')

    return agressor_relationship

def preprocess_relationship(value):
    
    if value == '*':
        return 0
    
    elif value == '':
        return 0

    else:
        
        try:
            return np.int8(value)
        
        except:
            
            try:
                return np.int8(float(value))
            
            except: # NaN variables
                return 0

In [110]:
%%time

# Corrigindo tipo das variáveis de relação

relationship_columns = ['REL_PAI','REL_MAE','REL_PAD','REL_CONJ','REL_EXCON',
                        'REL_NAMO','REL_EXNAM','REL_FILHO','REL_DESCO','REL_IRMAO',
                        'REL_CONHEC','REL_CUIDA','REL_PATRAO','REL_INST','REL_POL',
                        'REL_PROPRI','REL_OUTROS','REL_MAD']


for column in relationship_columns:

    # Convertendo valores para inteiros 
    raw_dataframe[column] = raw_dataframe[column].apply(preprocess_relationship)

Wall time: 21.5 s


In [ ]:
%%time

raw_dataframe['AGRESSOR'] = get_agressor_relationship(raw_dataframe)

In [ ]:
# Retirando dados omitidos
df  = raw_dataframe[raw_dataframe['AGRESSOR'] != 'N/A']

fig = px.histogram(df, x="AGRESSOR", title=f'Relação: Vítima x Agressor ({nrows} casos)')
fig.show()